# Difference Between OLTP HR Schema and Star Schema (DWH)

This notebook explains the difference between two schema diagrams:

1. **Star Schema (Data Warehouse model)** – optimized for analytics, with fact and dimension tables.
2. **Oracle HR OLTP Schema (3NF)** – optimized for transactional operations.

---

## 1. Purpose & Design

- **Star schema (DW):** Optimized for **reporting & analytics** (read-heavy, aggregations, dashboards).
- **OLTP HR schema:** Optimized for **transactions** (insert/update/delete, enforcing integrity).

---

## 2. Grain & Measures

- **Fact table (`employee_salary_fact`):** Grain = salary snapshot at a specific effective date. Measures include salary, bonus, total compensation.
- **OLTP:** Measures are attributes inside `employees` and history in `job_history`.

---

## 3. Keys

- **DW:** Uses surrogate keys (`RAW(16)`), enables Slowly Changing Dimensions (SCD).
- **OLTP:** Uses natural/business keys (`employee_id`, `department_id`, `job_id`).

---

## 4. Normalization vs. Denormalization

- **DW:** Denormalized – e.g., `location_dim` flattens location + country + region.
- **OLTP:** Normalized (3NF) – separate `locations`, `countries`, `regions`. Avoids redundancy.

---

## 5. History Handling

- **DW:** SCD (Type 2) captures multiple versions of employee/job/department over time.
- **OLTP:** `job_history` captures employment changes with start/end dates.

---

## 6. Time Dimension

- **DW:** `time_dim` provides year, quarter, month, fiscal attributes for reporting.
- **OLTP:** Stores raw dates (`hire_date`, `start_date`, `end_date`). No fiscal/calendar logic.

---

## 7. Querying

- **DW:** Simple queries – fact joins directly to dimensions. Example:

```sql
SELECT d.department_name, SUM(f.total_compensation)
FROM employee_salary_fact f
JOIN department_dim d ON f.surrogate_department_id = d.surrogate_department_id
GROUP BY d.department_name;
```

- **OLTP:** Complex joins across many tables. Example:

```sql
SELECT r.region_name, c.country_name, l.city, d.department_name, e.salary
FROM employees e
JOIN departments d ON e.department_id = d.department_id
JOIN locations l ON d.location_id = l.location_id
JOIN countries c ON l.country_id = c.country_id
JOIN regions r ON c.region_id = r.region_id;
```

---

## 8. ETL Mapping

- `employee_dim` ⟵ employees (+ derived attributes like full_name, tenure_band)
- `department_dim` ⟵ departments
- `job_dim` ⟵ jobs (+ job_category)
- `location_dim` ⟵ locations + countries + regions
- `time_dim` ⟵ generated calendar
- `employee_salary_fact` ⟵ employees + job_history

---

## 9. Summary

- **Star schema:** Surrogate keys, denormalized, SCD support, time dimension, optimized for analytics.
- **HR OLTP schema:** Business keys, normalized, integrity constraints, optimized for transactions.

---
